In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Activation,BatchNormalization,Input,Embedding,Dot,Dense,Flatten,Dropout,Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler,TensorBoard,EarlyStopping
from utils.common_functions import read_yaml


from wordcloud import WordCloud
%matplotlib inline

### Reading ANIMELIST.CSV

In [ ]:
INPUT_DIR = os.path.join("..","artifacts","raw")

In [ ]:
rating_df = pd.read_csv(INPUT_DIR+'/animelist_filtered.csv',low_memory=True,usecols=['user_id','anime_id','rating'])

In [ ]:
len(rating_df)

### Data Processing

In [ ]:
min_rating = np.min(rating_df['rating'])-1
max_rating = np.max(rating_df['rating'])
avg_rating = np.mean(rating_df['rating'])

In [ ]:
avg_rating

In [ ]:
rating_df['rating']= rating_df['rating'].apply(lambda x:(x-min_rating)/(max_rating-min_rating)).values.astype('float64')

In [ ]:
user_id = rating_df['user_id'].unique().tolist()
user2user_encoder = {x: i for i,x in enumerate(user_id)}
user2user_decoder = {i: x for i,x in enumerate(user_id)}
rating_df['user'] = rating_df['user_id'].map(user2user_encoder)

In [ ]:
n_users = len(user2user_encoder)

### 16000 users!
user2user_encoder { 12321 : 83}
user2user_decoder { 83 : 12321}

In [ ]:
len(user2user_encoder)

In [ ]:
anime_id = rating_df['anime_id'].unique().tolist()
anime2anime_encoder = {x: i for i,x in enumerate(anime_id)}
anime2anime_decoder = {i: x for i,x in enumerate(anime_id)}
rating_df['anime'] = rating_df['anime_id'].map(anime2anime_encoder)

In [ ]:
n_animes = len(anime2anime_encoder)

In [ ]:
n_animes

In [ ]:
n_users

In [ ]:
len(anime2anime_encoder)

In [ ]:
rating_df

In [ ]:
rating_df = rating_df.sample(frac=1,random_state=43).reset_index(drop=True)

In [ ]:
X = rating_df[['user','anime']].values
y = rating_df['rating']

In [ ]:
test_size = 100000
train_indices = rating_df.shape[0] - test_size

In [ ]:
X_train , X_test , y_train , y_test = (
    X[:train_indices],
    X[train_indices :],
    y[:train_indices],
    y[train_indices :]
)

In [ ]:
X_test

In [ ]:
X_train_array = [X_train[:,0],X_train[:,1]]
X_test_array = [X_test[:,0],X_test[:,1]]

### MODEL ARTITECTURE

In [ ]:
def RecommenderNet(n_users, n_animes, embedding_size=32):
    user = Input(name='user', shape=[1])
    anime = Input(name='anime', shape=[1])

    user_embedding = Embedding(
        input_dim=n_users,
        output_dim=embedding_size,
        embeddings_regularizer=l2(1e-6),
        name='user_embedding'
    )(user)

    anime_embedding = Embedding(
        input_dim=n_animes,
        output_dim=embedding_size,
        embeddings_regularizer=l2(1e-6),
        name='anime_embedding'
    )(anime)

    user_vec = Flatten(name='flatten')(user_embedding)
    anime_vec = Flatten(name='flatten_1')(anime_embedding)

    x = Concatenate(name='concatenate')([user_vec, anime_vec])

    x = Dense(128, kernel_initializer='he_normal', name='dense')(x)
    x = BatchNormalization(name='batch_normalization')(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Dense(64, kernel_initializer='he_normal', name='dense_1')(x)
    x = BatchNormalization(name='batch_normalization_1')(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Dense(1, activation='sigmoid', name='dense_2')(x)

    model = Model(inputs=[user, anime], outputs=x)
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=1e-3),
        metrics=['mae', 'mse']
    )
    
    return model

In [ ]:
# === Training hyperparams ===
batch_size = 512
learning_rate = 1e-3   # constant Adam LR

In [ ]:

# === Build and compile model ===
recomm_model = RecommenderNet(n_users, n_animes)
recomm_model.compile(
    loss="binary_crossentropy",
    optimizer=Adam(learning_rate=learning_rate),
    metrics=["mae", "mse"]
)


In [ ]:
# === Checkpoint: save full model (.keras) ===
checkpoint_filepath = 'best_recommender_model.weights.h5'
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

In [ ]:
# === Early stopping ===
early_stopping = EarlyStopping(
    patience=3,
    monitor='val_loss',
    mode='min',
    restore_best_weights=True
)

In [ ]:
# === Load Model ===
history = recomm_model.load_weights(checkpoint_filepath)

In [ ]:
print(len(X_train_array[0]))

In [ ]:
# === Train ===

# recomm_model.fit(
#     x=X_train_array,
#     y=y_train,
#     batch_size=batch_size,
#     epochs=40,
#     validation_data=(X_test_array, y_test),
#     callbacks=[model_checkpoint, early_stopping]
# )

In [ ]:
# metrics = ["loss", "mae", "mse"]
# fig, axes = plt.subplots(len(metrics), 1, figsize=(8, len(metrics) * 4))

# for i, metric in enumerate(metrics):
#     ax = axes[i]
#     ax.plot(recomm_model.history.history[metric][0:-2], marker="o", label=f"train {metric}")
#     ax.plot(recomm_model.history.history[f"val_{metric}"][0:-2], marker="o", label=f"test {metric}")
#     ax.set_title(f"Model {metric.capitalize()}")
#     ax.set_ylabel(metric.capitalize())
#     ax.set_xlabel("Epoch")
#     ax.legend(loc="upper left")
#     ax.grid(True)

# plt.tight_layout()
# plt.show()

In [ ]:
def extract_weights(name,model):
    weight_layer = model.get_layer(name)
    weights = weight_layer.get_weights()[0]
    weights = weights/np.linalg.norm(weights,axis=1).reshape((-1,1))
    return weights

In [ ]:
anime_weights = extract_weights('anime_embedding',recomm_model)
user_weights = extract_weights('user_embedding',recomm_model)


### READING ANIME.CSV

In [ ]:
df = pd.read_csv(INPUT_DIR+'/anime.csv',low_memory=True)
df = df.replace("Unknown",np.nan)


In [ ]:
def getAnimeName(anime_id):
    try:
        name = df[df.anime_id == anime_id].anime_name.values[0]
        if name is np.nan:
            name = df[df.anime_id == anime_id].Name.values[0]
    except:
        print("Error")
    return name

In [ ]:
df['anime_id']=df['MAL_ID']
df['anime_name']=df['English name']
df['score']=df['Score']
df['genres']=df['Genres']
df['episodes']=df['Episodes']
df['type']=df['Type']
df['premiered']=df['Premiered']
df['members']=df['Members']
df['anime_name'] = df.anime_id.apply(lambda x :getAnimeName(x))

In [ ]:
df.sort_values(by=['Score'],inplace=True,ascending=False,na_position='last')

In [ ]:
df = df[['anime_id','anime_name','score','genres','episodes','type','premiered','members']]

df.head()

In [ ]:
def getAnimeFrame(anime,df):
    if isinstance(anime,int):return df[df.anime_id == anime]
    if isinstance(anime,str):return df[df.anime_name == anime]

In [ ]:
getAnimeFrame('Attack on Titan Final Season',df)

### READING ANIMESYNOPSIS.CSV

In [ ]:
cols = ['MAL_ID','Name','Genres','sypnopsis']
syn_df = pd.read_csv(INPUT_DIR+"/anime_with_synopsis.csv")
syn_df['anime_id']=syn_df['MAL_ID']
syn_df['anime_name']=syn_df['Name']
syn_df['syn']=syn_df['sypnopsis']
syn_df['genres']=syn_df['Genres']
syn_df = syn_df[['anime_id','anime_name','genres','syn']]

In [ ]:
syn_df.head(2)

In [ ]:
def getAnimeSyn(anime,df):
    if isinstance(anime,int):return df[df.anime_id == anime].syn.values[0]
    if isinstance(anime,str):return df[df.anime_name == anime].syn.values[0]

In [ ]:
getAnimeSyn('Steins;Gate',syn_df)

In [ ]:
def getAnimeName(df,anime_id):
    name = df[df.anime_id == anime_id].anime_name.values[0]
    if name is np.nan: name = df[df.anime_id == anime_id].Name.values[0]
    return name

In [ ]:
def load_anime_df(df,syn_df=None):
        df.replace("Unknown", np.nan).rename(
                columns={
                    "MAL_ID": "anime_id",
                    "English name": "anime_name",
                    "Score": "score",
                    "Genres": "genres",
                    "Episodes": "episodes",
                    "Type": "type",
                    "Premiered": "premiered",
                    "Members": "members",
                }
            )
        if syn_df is not None:
            syn_df = syn_df.rename(
                columns={
                    "MAL_ID": "anime_id",
                    "Name": "anime_name",
                    "Genres": "genres",
                    "sypnopsis": "syn"
                }
            )
        return df.merge(syn_df[["anime_id", "syn"]], on="anime_id", how="left")

In [ ]:
anime_df = load_anime_df(df,syn_df=syn_df)[['anime_id','anime_name','score','genres','episodes','type','premiered','members','syn']]
anime_df.sort_values(by=['score'],inplace=True,ascending=False,na_position='last')

In [ ]:
anime_df

In [ ]:
anime_df

### CONTENT BASED RECOMMENDATION

In [ ]:
pd.set_option('max_colwidth',None)

In [ ]:
df['genres'] = df['genres'].fillna('')
df['genre_list'] = df['genres'].str.split(', ').apply(list)

In [ ]:
df.head(3)

In [ ]:
def find_similar_animes(name, anime_weights,
                        anime2anime_encoder, anime2anime_decoder,
                        df, syn_df, n=10, return_dist=False, negative=False, 
                        consider_genres=True, genre_weight=0.2):
    
    index = getAnimeFrame(name, df).anime_id.values[0]
    encoded_index = anime2anime_encoder.get(index)
    weights = anime_weights

    query_vec = weights[encoded_index]
    dists = np.dot(weights, query_vec) / (np.linalg.norm(weights, axis=1) * np.linalg.norm(query_vec) + 1e-9)

    if consider_genres:
        query_anime_frame = df[df['anime_id'] == index]
        if query_anime_frame.empty:
            query_genres = set()
        else:
            genre_list = query_anime_frame['genre_list'].values[0]
            query_genres = set(genre_list) if genre_list and len(genre_list) > 0 else set()
        
        sorted_dists = np.argsort(dists) if negative else np.argsort(dists)[::-1]
        top_candidates = sorted_dists[:min(200, len(sorted_dists))]
        
        final_scores = []
        
        for candidate_idx in top_candidates:
            decoded_id = anime2anime_decoder.get(candidate_idx)
            if decoded_id is None or decoded_id == index:
                continue
            
            candidate_frame = df[df['anime_id'] == decoded_id]
            if candidate_frame.empty:
                continue
                
            genre_list = candidate_frame['genre_list'].values[0]
            candidate_genres = set(genre_list) if genre_list and len(genre_list) > 0 else set()
            
            if len(candidate_genres) > 0 and len(query_genres) > 0:
                matching_genres = len(query_genres.intersection(candidate_genres))
                genre_overlap = matching_genres / len(candidate_genres)
            else:
                genre_overlap = 0
            
            cosine_score = dists[candidate_idx]
            final_score = cosine_score * (1 - genre_weight) + (genre_overlap**1) * genre_weight
            
            final_scores.append({
                'candidate_idx': candidate_idx,
                'decoded_id': decoded_id,
                'cosine_similarity': cosine_score,
                'genre_overlap': genre_overlap,
                'final_score': final_score
            })
        
        final_scores.sort(key=lambda x: x['final_score'], reverse=not negative)
        
    else:
        sorted_dists = np.argsort(dists) if negative else np.argsort(dists)[::-1]
        final_scores = []
        for idx in sorted_dists[:n+50]:
            decoded_id = anime2anime_decoder.get(idx)
            if decoded_id is None or decoded_id == index:
                continue
            final_scores.append({
                'candidate_idx': idx,
                'decoded_id': decoded_id,
                'final_score': dists[idx],
                'cosine_similarity': dists[idx],
                'genre_overlap': 0
            })

    if return_dist:
        return dists, [item['candidate_idx'] for item in final_scores[:n]]
    
    rank = 1
    similarity_arr = []
    
    for item in final_scores:
        if rank > n:
            break
            
        decoded_id = item['decoded_id']
        syn = getAnimeSyn(decoded_id, syn_df)
        anime_frame = df[df['anime_id'] == decoded_id]
        
        if anime_frame.empty:
            continue
            
        anime_name = anime_frame['anime_name'].values[0]
        genre = anime_frame['genres'].values[0]
        
        if not np.isnan(item['final_score']):
            similarity_arr.append({
                'rank': rank,
                'anime_id': decoded_id,
                'name': anime_name,
                'similarity': item['cosine_similarity'],
                'genre_overlap': item.get('genre_overlap', 0),
                'final_score': item['final_score'],
                'genre': genre,
                'syn': syn
            })
            rank += 1

    Frame = pd.DataFrame(similarity_arr)
    return Frame.head(n)

In [ ]:
df.head(10)['genres']

In [ ]:
getAnimeFrame(45, df)

In [ ]:
anime2anime_encoder

In [99]:
def generate_decoders(encoder):
    decoder  = {}
    for key,value in encoder.items():
        decoder[value] = key
    return decoder

In [100]:
anime2anime_encoder[17831]

300

In [101]:
decoder = generate_decoders(anime2anime_encoder)
decoder[300]

17831

In [ ]:
generate_decoders()

In [ ]:
df

In [ ]:
result = find_similar_animes(
    name = 'One Piece',
    anime_weights = anime_weights,
    anime2anime_encoder=anime2anime_encoder,
    anime2anime_decoder=anime2anime_decoder,
    df=df,
    syn_df=syn_df,
    consider_genres=True,
    genre_weight=0.1,
    n=10
)

In [ ]:
result.head(10)['name']

### USER BASED RECOMMENDAION

In [ ]:
def find_similar_users(item_input, user_weights, user2user_encoder, user2user_decoder, n=10, return_dist=False, negative=False):
    encoded_index = user2user_encoder.get(int(item_input))
    dists = np.dot(user_weights, user_weights[encoded_index])
    nan_mask = np.isnan(dists)
    valid_indices = np.where(~nan_mask)[0]
    nan_indices = np.where(nan_mask)[0]
    valid_sorted = valid_indices[np.argsort(dists[valid_indices])[::-1] if not negative else np.argsort(dists[valid_indices])]
    closests = np.concatenate([valid_sorted, nan_indices])
    if return_dist:return dists, closests[:n+1]
    
    similarity_data = [
        {'similar_user': user2user_decoder.get(close), 'similarity': dists[close]}
        for close in closests[:n+1]
        if close != encoded_index]
    
    return pd.DataFrame(similarity_data)

In [ ]:
find_similar_users(10409,
                   user_weights,
                   user2user_encoder,
                   user2user_decoder
                   )

In [ ]:
def showWordCloud(all_genres):
    genres_cloud = WordCloud(width=700,height=400,background_color='white',colormap='gnuplot').generate_from_frequencies(all_genres)
    plt.figure(figsize=(10,8))
    plt.imshow(genres_cloud,interpolation="bilinear")
    plt.axis("off")
    plt.show()

In [ ]:
from collections import defaultdict

In [ ]:
df.head(1)

In [ ]:
def getFavGenre(frame , plot=False):
    cleaned_frame = frame.dropna(inplace=False)
    all_genres = defaultdict(int)

    genres_list = []
    for genres in cleaned_frame["genres"]:
        if isinstance(genres,str):
            for genre in genres.split(','):
                genres_list.append(genre)
                all_genres[genre.strip()] += 1

    if plot:
        showWordCloud(all_genres)
    
    return genres_list

In [ ]:
getFavGenre(df,plot=True)

In [ ]:
def get_user_preferences(user_id,rating_df,df,verbose = 0,plot=False):
    animes_watched_by_user = rating_df[rating_df.user_id == user_id]
    animes_watched_by_user = animes_watched_by_user.dropna(subset=['rating'])
    user_rating_percentile = np.percentile(animes_watched_by_user.rating, 75)
    animes_watched_by_user = animes_watched_by_user[animes_watched_by_user.rating >= user_rating_percentile]
    top_animes_user = animes_watched_by_user.sort_values(by='rating', ascending=False).anime_id.values
    anime_df_rows = df[df["anime_id"].isin(top_animes_user)]
    if plot: getFavGenre(anime_df_rows,plot)
    return anime_df_rows[["anime_name", "genres"]]


In [ ]:
df

In [ ]:
rating_df

In [ ]:
get_user_preferences(10409,rating_df,df,plot=True)

In [ ]:
def get_user_recommendations(similar_users, user_pref, df, syn_df, rating_df, n=10):
    anime_list = []
    
    for user_id in similar_users.similar_user.values:
        pref_list = get_user_preferences(int(user_id), rating_df, df)
        pref_list = pref_list[~pref_list.anime_name.isin(user_pref.anime_name.values)]
        if not pref_list.empty:
            anime_list.extend(pref_list.anime_name.values)
    if not anime_list:
        return pd.DataFrame()
    
    anime_counts = pd.Series(anime_list).value_counts().head(n)
    recommendations = [
        {
            "n": count,
            "anime_name": anime_name,
            "genres": (frame := getAnimeFrame(anime_name, df)).genres.values[0]
        }
        for anime_name, count in anime_counts.items()
        if isinstance(anime_name, str)
    ]
    return pd.DataFrame(recommendations)

In [ ]:
similar_users = find_similar_users(10409,user_weights,user2user_encoder,user2user_decoder)

In [ ]:
user_pref = get_user_preferences(10409 , rating_df, df , plot=False)

In [ ]:
rating_df

In [ ]:
similar_users

In [ ]:
get_user_recommendations(similar_users,user_pref,df, syn_df,rating_df,n=5)

In [ ]:
def hybrid_recommendation(user_id, user_weight=0.5, content_weight=0.5):
    similar_users = find_similar_users(user_id, user_weights, user2user_encoder, user2user_decoder)
    user_pref = get_user_preferences(user_id, rating_df, df)
    user_recommended_animes = get_user_recommendations(similar_users, user_pref, df, syn_df, rating_df)
    user_list = user_recommended_animes["anime_name"].tolist()
    
    content_list = [
        similar_anime
        for anime in user_list
        for similar_animes in [find_similar_animes(anime, anime_weights, anime2anime_encoder, anime2anime_decoder, df, syn_df)]
        if similar_animes is not None and not similar_animes.empty
        for similar_anime in similar_animes["name"].tolist()
    ]
    
    from collections import Counter
    combined_scores = Counter({anime: user_weight for anime in user_list})
    combined_scores.update({anime: content_weight for anime in content_list})
    return combined_scores.most_common(10)

In [ ]:
def create_temp_user_profile(anime_ratings_dict, df):
    temp_profile = []
    for anime_name, rating in anime_ratings_dict.items():
        anime_frame = getAnimeFrame(anime_name, df)
        if not anime_frame.empty:
            anime_id = anime_frame.anime_id.values[0]
            temp_profile.append({
                'user_id': -1, 
                'anime_id': anime_id,
                'anime_name': anime_name,
                'rating': rating
            })
    
    return pd.DataFrame(temp_profile)


In [ ]:
def build_user_inf_model(recomm_model, embedding_size=32):
    
    user_vec = Input(shape=(embedding_size,), name='user_vec')
    anime = Input(shape=(1,), name='anime')
    
    anime_emb = recomm_model.get_layer('anime_embedding')(anime)
    anime_vec = Flatten(name='flatten_anime')(anime_emb)
    
    x = Concatenate(name='concat')([user_vec, anime_vec])
    
    x = recomm_model.get_layer('dense')(x)
    x = recomm_model.get_layer('batch_normalization')(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    
    x = recomm_model.get_layer('dense_1')(x)
    x = recomm_model.get_layer('batch_normalization_1')(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    
    x = recomm_model.get_layer('dense_2')(x)
    
    inf_model = Model(inputs=[user_vec, anime], outputs=x)
    return inf_model

In [132]:
def get_temp_user_embedding(temp_user_profile, inf_model, anime2anime_encoder, embedding_size=32, min_rating=None, max_rating=None, epochs=50, lr=0.01):
    import tensorflow as tf
    import numpy as np
    
    anime_list = []
    rating_list = []
    for _, row in temp_user_profile.iterrows():
        anime_id = row['anime_id']
        encoded_anime = anime2anime_encoder.get(anime_id)
        if encoded_anime is not None:
            anime_list.append(encoded_anime)
            normalized_rating = (row['rating'] - min_rating) / (max_rating - min_rating) if min_rating is not None and max_rating is not None else row['rating'] / 10.0
            rating_list.append(normalized_rating)
    
    if not anime_list:
        return np.zeros(embedding_size)
    
    anime_indices = tf.constant(anime_list, dtype=tf.int32)[:, tf.newaxis] 
    targets = tf.constant(rating_list, dtype=tf.float32)
    
    user_emb_var = tf.Variable(tf.random.normal([embedding_size], stddev=0.05))
    optimizer = tf.optimizers.Adam(learning_rate=lr)
    loss_fn = tf.keras.losses.BinaryCrossentropy()
    
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            user_vec_input = tf.tile(user_emb_var[tf.newaxis, :], [len(anime_list), 1])
            preds = inf_model([user_vec_input, anime_indices])
            preds = tf.squeeze(preds)
            loss = loss_fn(targets, preds)
        grads = tape.gradient(loss, [user_emb_var])
        optimizer.apply_gradients(zip(grads, [user_emb_var]))
    
    temp_emb = user_emb_var.numpy()
    norm = np.linalg.norm(temp_emb)
    return temp_emb / norm if norm > 0 else temp_emb

In [133]:
def find_similar_users_for_temp(temp_embedding, user_weights, user2user_decoder, n=10):
    import numpy as np
    if np.all(temp_embedding == 0):
        print("Warning: Temp user embedding is zero; returning empty user list.")
        return pd.DataFrame(columns=['similar_user', 'similarity'])
    similarities = np.dot(user_weights, temp_embedding)
    nan_mask = np.isnan(similarities)
    valid_indices = np.where(~nan_mask)[0]
    valid_sorted = valid_indices[np.argsort(similarities[valid_indices])[::-1]]
    similar_users_data = []
    for i, user_idx in enumerate(valid_sorted[:n]):
        user_id = user2user_decoder.get(user_idx)
        similarity = similarities[user_idx]
        similar_users_data.append({
            'similar_user': user_id,
            'similarity': similarity
        })
    return pd.DataFrame(similar_users_data)

In [134]:
def hybrid_recommendation_for_temp_user(temp_similar_users, temp_user_profile, user_weight=0.5, content_weight=0.5,n=10):
    user_recommended_animes = get_user_recommendations(temp_similar_users, temp_user_profile, df, syn_df, rating_df)
    user_list = user_recommended_animes["anime_name"].tolist()
    
    content_list = [
        similar_anime
        for anime in user_list
        for similar_animes in [find_similar_animes(anime, anime_weights, anime2anime_encoder, anime2anime_decoder, df, syn_df)]
        if similar_animes is not None and not similar_animes.empty
        for similar_anime in similar_animes["name"].tolist()
    ]
    
    from collections import Counter
    combined_scores = Counter({anime: user_weight for anime in user_list})
    combined_scores.update({anime: content_weight for anime in content_list})
    return combined_scores.most_common(n)




In [135]:
def recommend_for_temp_user(anime_ratings_dict, n_recommendations=10, user_weight=0.5, content_weight=0.5):
    temp_user_profile = create_temp_user_profile(anime_ratings_dict, df)
    inf_model = build_user_inf_model(recomm_model)
    temp_embedding = get_temp_user_embedding(temp_user_profile, inf_model,anime2anime_encoder, min_rating=min_rating, max_rating=max_rating)
    similar_users = find_similar_users_for_temp(temp_embedding, user_weights, user2user_decoder, n=10)
    hybrid_results = hybrid_recommendation_for_temp_user(similar_users, temp_user_profile, user_weight, content_weight)
    
    if not hybrid_results:
        return "Sorry, Couldn't find any anime matching with you!"
    
    max_score = hybrid_results[0][1] if hybrid_results else 1
    
    final_recommendations = []
    for anime_name, score in hybrid_results[:n_recommendations]:
        anime_frame = getAnimeFrame(anime_name, df)
        if not anime_frame.empty:
            probability = min((score / max_score) * 0.8, 0.8)
            final_recommendations.append({
                'anime_name': anime_name,
                'genres': anime_frame.genres.values[0],
                'hybrid_score': score,
                'probability': probability
            })
    return pd.DataFrame(final_recommendations)

In [136]:
user_ratings = {
    'Attack on Titan': 9,
    'Death Note': 8,
    'One Piece': 1,
    'Naruto': 6,
    'Monster' : 2
}

In [137]:
recommendations = recommend_for_temp_user(user_ratings,user_weight=0.3,content_weight=0.7)

In [138]:
recommendations

,anime_name,genres,hybrid_score,probability
0,Bakuman.,"Comedy, Drama, Romance, Shounen",1.0,0.80
1,Nisekoi:False Love,"Harem, Comedy, Romance, School, Shounen",1.0,0.80
2,Noragami Aragoto,"Action, Adventure, Comedy, Supernatural, Shounen",1.0,0.80
3,Aldnoah.Zero,"Action, Military, Sci-Fi, Mecha",1.0,0.80
4,GATE,"Action, Military, Adventure, Fantasy",1.0,0.80
5,Is It Wrong to Try to Pick Up Girls in a Dungeon?,"Action, Adventure, Comedy, Romance, Fantasy",1.0,0.80
6,Nintama Rantarou Movie: Ninjutsu Gakuen Zenin Shutsudou! no Dan,"Comedy, Shounen",0.7,0.56
7,Piercing I,Drama,0.7,0.56
8,Gakkyuu Ou Yamazaki Specials,Comedy,0.7,0.56
9,Haikyu!! 3rd Season,"Comedy, Sports, Drama, School, Shounen",0.7,0.56


In [139]:
recomm_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ anime (InputLayer)  │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 32)     │    561,536 │ user[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ anime_embedding     │ (None, 1, 32)     │    548,480 │ anime[0][0]       │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 32)        │          0 │ user_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 32)        │          0 │ anime_embedding[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64)        │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │      8,320 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128)       │        512 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 128)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 64)        │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         65 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,127,425 (4.30 MB)

 Trainable params: 1,127,041 (4.30 MB)

 Non-trainable params: 384 (1.50 KB)